In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import json
from tqdm import tqdm
import pandas as pd
import os

In [ ]:
regions = ['chungcheong','jeju','jeonla','kangwon','kyeongsang']

In [ ]:
%cd /content/drive/MyDrive/OSSP

/content/drive/MyDrive/OSSP


In [ ]:
files = glob.glob('data/'+regions[-1]+'/*.zip')
files.sort()

In [ ]:
files

## json to df

In [ ]:
def get_lst(filename):
    try:
        with open(filename,'r',encoding='utf-8-sig') as f:
            data=json.load(f)
        f.close()
    except:
        print(filename)
    standard = []
    dialect = []
    mp_d = []
    mp_s = []
    mp_idx = []
    for d in data['utterance']:
        # 다른 형태만 가져옴
        if d['standard_form'] != d['dialect_form']:
            standard.append(d['standard_form'])
            dialect.append(d['dialect_form'])
            tmp = []
            tmp2 = []
            tmp3 = []
            for el in d['eojeolList']:
                if el['isDialect']:
                    tmp.append(el['eojeol'])
                    tmp2.append(el['standard'])
                    tmp3.append(el['id'])
            mp_d.append('_'.join(tmp))
            mp_s.append('_'.join(tmp2))
            mp_idx.append(' '.join([str(t) for t in tmp3]))
    return standard, dialect, mp_d, mp_s, mp_idx

In [ ]:
def extract_df(file_path, reg, tp):
    standard_lst, dial_lst, mp_d_lst, mp_s_lst, mp_idx_lst = [], [], [], [], []
    for fn in tqdm(file_path):
        try:
            standard, dialect, mp_d, mp_s, mp_idx = get_lst(fn)
        except:
            pass
        standard_lst+=standard
        dial_lst+=dialect
        mp_d_lst+=mp_d
        mp_s_lst+=mp_s
        mp_idx_lst+=mp_idx

    df=pd.DataFrame(columns=['standard','dialect','mp_d','mp_s','mp_idx'])
    df['standard'] = standard_lst
    df['dialect'] = dial_lst
    df['mp_d'] = mp_d_lst
    df['mp_s'] = mp_s_lst
    df['mp_idx'] = mp_idx_lst
    save_path = 'data/{}/{}/{}_data.tsv'.format(reg,tp,tp)
    df.to_csv(save_path,index=False,sep='\t')
    print('saved in {}'.format(save_path))

In [ ]:
regions = ['jeju','jeonla','kangwon','kyeongsang' , 'chungcheong']
types= ['train','test']

In [ ]:
for r in regions:
    for t in types:
        files = glob.glob('[라벨]제주도_학습용데이터_3/{}.json'.format(r,t))
        print(r,t)
        extract_df(files, r, t)

## remove raw files

In [ ]:
import os

In [ ]:
regions = ['chungcheong','jeju','jeonla','kangwon','kyeongsang']
types= ['train','test']

In [ ]:
for r in regions:
    for t in types:
        for f in glob.glob('data/{}/{}/*.json'.format(r,t)):
            os.remove(f)
        for f in glob.glob('data/{}/{}/*.txt'.format(r,t)):
            os.remove(f)